# Discovery

By now, world population is in constant increase. The more the people, the more the food we need that translate in an increase of use of natural resources.


But which are the product we produce that exploit most resources? Which countries contribute to the exploitation of natural resources?

**Goal**: Discovering which are the top 10 products that use more water, need more land and emit more gas. Then discovering which countries are the most producer of each category


# Data Selection

In [72]:
import pandas as pd
import numpy as np
import sidetable 
from dataprep.clean import clean_country
from dataprep.clean import clean_df
from dataprep.clean import validate_lat_long
import country_converter as coco
import plotly 
import plotly.express as px
import plotly.io as pio 


In [73]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.display.max_colwidth = 100
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [74]:
fao_df = pd.read_csv('FAO.csv', encoding='latin-1')
food_pr_df = pd.read_csv('Food_Production.csv')

# Data Cleaning

### FAO Data

In [75]:
fao_df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,"1,928.00","1,904.00","1,666.00","1,950.00","2,001.00","1,808.00","2,053.00","2,045.00","2,154.00","1,819.00","1,963.00","2,215.00","2,310.00","2,335.00","2,434.00","2,512.00","2,282.00","2,454.00","2,443.00","2,129.00","2,133.00","2,068.00","1,994.00","1,851.00","1,791.00","1,683.00","2,194.00","1,801.00","1,754.00","1,640.00","1,539.00","1,582.00","1,840.00","1,855.00","1,853.00","2,177.00","2,343.00","2,407.00","2,463.00","2,600.00","2,668.00","2,776.00","3,095.00","3,249.00","3,486.00","3,704.00","4,164.00","4,252.00","4,538.00","4,605.00","4,711.00",4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.00,183.00,182.00,220.00,220.00,195.00,231.00,235.00,238.00,213.00,205.00,233.00,246.00,246.00,255.00,263.00,235.00,254.00,270.00,259.00,248.00,217.00,217.00,197.00,186.00,200.00,193.00,202.00,191.00,199.00,197.00,249.00,218.00,260.00,319.00,254.00,326.00,347.00,270.00,372.00,411.00,448.00,460.00,419.00,445.00,546.00,455.00,490.00,415.00,442.00,476.00,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.00,76.00,76.00,76.00,76.00,75.00,71.00,72.00,73.00,74.00,71.00,70.00,72.00,76.00,77.00,80.00,60.00,65.00,64.00,64.00,60.00,55.00,53.00,51.00,48.00,46.00,46.00,47.00,46.00,43.00,43.00,40.00,50.00,46.00,41.00,44.00,50.00,48.00,43.00,26.00,29.00,70.00,48.00,58.00,236.00,262.00,263.00,230.00,379.00,315.00,203.00,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.00,237.00,237.00,238.00,238.00,237.00,225.00,227.00,230.00,234.00,223.00,219.00,225.00,240.00,244.00,255.00,185.00,203.00,198.00,202.00,189.00,174.00,167.00,160.00,151.00,145.00,145.00,148.00,145.00,135.00,132.00,120.00,155.00,143.00,125.00,138.00,159.00,154.00,141.00,84.00,83.00,122.00,144.00,185.00,43.00,44.00,48.00,62.00,55.00,60.00,72.00,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.00,210.00,214.00,216.00,216.00,216.00,235.00,232.00,236.00,200.00,201.00,216.00,228.00,231.00,234.00,240.00,228.00,234.00,228.00,226.00,210.00,199.00,192.00,182.00,173.00,170.00,154.00,148.00,137.00,144.00,126.00,90.00,141.00,150.00,159.00,108.00,90.00,99.00,72.00,35.00,48.00,89.00,63.00,120.00,208.00,233.00,249.00,247.00,195.00,178.00,191.00,200,200


In [76]:
fao_df.shape

(21477, 63)

#### Checking columns dtypes

In [77]:
fao_dtypes, clean_fao = clean_df(fao_df, clean_header=False, standardize_missing_values='ignore', downcast_memory=False)

Data Type Detection Report:
	These data types are supported by DataPrep to clean: ['country', 'coordinate', 'address']


In [78]:
fao_dtypes

,semantic_data_type,atomic_data_type
Area Abbreviation,country,string
Area Code,integer,integer
Area,country,string
Item Code,integer,integer
Item,string,string
Element Code,integer,integer
Element,string,string
Unit,address,string
latitude,coordinate,floating
longitude,coordinate,floating


changing *Y2012* and *Y2013* data type from int to float

In [79]:
clean_fao[['Y2012', 'Y2013']] = clean_fao[['Y2012', 'Y2013']].astype(float)

#### Checking column names and eventually fixing them

In [80]:
clean_fao.columns

Index(['Area Abbreviation', 'Area Code', 'Area', 'Item Code', 'Item',
       'Element Code', 'Element', 'Unit', 'latitude', 'longitude', 'Y1961',
       'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969',
       'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977',
       'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985',
       'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993',
       'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001',
       'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009',
       'Y2010', 'Y2011', 'Y2012', 'Y2013'],
      dtype='object')

In [81]:
clean_fao.rename(columns={n:n[1:] for n in clean_fao.columns if 'Y' in n}, inplace=True) 

#### Checking for duplicates

In [82]:
print(f'Number of rows before dropping duplicates: {clean_fao.shape[0] :>6}')
clean_fao.drop_duplicates(keep='first')
print(f'Number of rows after dropping duplicates: {clean_fao.shape[0] :>7}')

Number of rows before dropping duplicates:  21477
Number of rows after dropping duplicates:   21477


Counting the number of rows before and after checking for duplicates. No duplicates were found

#### Checking qualitative columns values

In [83]:
years = clean_fao.columns[clean_fao.columns.get_loc('1961'):]
clean_fao.stb.counts(exclude=['number'])

,count,unique,most_freq,most_freq_count,least_freq,least_freq_count
Unit,21477,1,1000 tonnes,21477,1000 tonnes,21477
Element,21477,2,Food,17528,Feed,3949
Item,21477,115,Milk - Excluding Butter,558,"Meat, Aquatic Mammals",3
Area Abbreviation,21477,169,CHN,541,LSO,75
Area,21477,174,Spain,150,Lesotho,75


In [84]:
to_check = ['Area', 'Area Abbreviation', 'Item']
for col in to_check:
    print(f'{col}\n{sorted(clean_fao[col].unique())}', end=f'\n{"-"*250}\n')
    

Area
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China, Hong Kong SAR', 'China, Macao SAR', 'China, Taiwan Province of', 'China, mainland', 'Colombia', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', "Côte d'Ivoire", "Democratic People's Republic of Korea", 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Icel

#### Fixing Area column values

After a close look, some of the Area names are not correct. let's correct them:

With the function clean_country *Taiwan*, *Macau* and *Hong Kong* would fall under *China*, let's remove it from those country before applying the function

In [85]:
clean_fao['Area'] = clean_fao['Area'].apply(lambda x: 'Taiwan' if 'Taiwan' in x else x)
clean_fao['Area'] = clean_fao['Area'].apply(lambda x: 'Hong Kong' if 'Hong Kong' in x else x)
clean_fao['Area'] = clean_fao['Area'].apply(lambda x: 'Macau' if 'Macao' in x else x)

Cleaning country names with clean_country

In [86]:
clean_fao = clean_country(clean_fao, 'Area', input_format=('name', 'official'), output_format='official', fuzzy_dist=2, inplace=True, errors='raise')

  0%|          | 0/9 [00:00<?, ?it/s]

Country Cleaning Report:
	17246 values cleaned (80.3%)
Result contains 21477 (100.0%) values in the correct format and 0 null values (0.0%)


 *Area Abbreviation* and the *Area* count do not match. Let's investigate why

In [87]:
# checking if there are more than 1 country with the same area code
a_code_unique = clean_fao.groupby('Area Abbreviation')['Area_clean'].nunique()
a_code_unique[a_code_unique > 1]

Area Abbreviation
AZE    2
CHN    4
THA    2
Name: Area_clean, dtype: int64

As suspected, 3 *Area Abbreviation* are associated with more than 1 country. Let's find out which countries fall under the same code

In [88]:
cond = clean_fao['Area Abbreviation'].isin(['AZE', 'THA', 'CHN'])
clean_fao.loc[cond].groupby('Area Abbreviation')['Area_clean'].unique()

Area Abbreviation
AZE                        [Republic of Azerbaijan, Commonwealth of the Bahamas]
CHN    [Hong Kong SAR, Macau SAR, People's Republic of China, Republic of China]
THA                                 [Kingdom of Thailand, Republic of Macedonia]
Name: Area_clean, dtype: object

The result of the investigation evidenciate that *Bahamas*, *Macedonia*, *Taiwan*, *Macau* and *Hong Kong* fall under the wrong *Area abbreviation* code.

Fixing the wrong Area Abbreviation values

In [89]:
cond = clean_fao['Area_clean'] == 'Bahamas'
clean_fao.loc[cond, 'Area Abbreviation'] = 'BHS'

cond = clean_fao['Area_clean'] == 'Hong Kong'
clean_fao.loc[cond, 'Area Abbreviation'] = 'HKG'

cond = clean_fao['Area_clean'] == 'Macau'
clean_fao.loc[cond, 'Area Abbreviation'] = 'MAC'

cond = clean_fao['Area_clean'] == 'Taiwan'
clean_fao.loc[cond, 'Area Abbreviation'] = 'TWN'

cond = clean_fao['Area_clean'] == 'Macedonia'
clean_fao.loc[cond, 'Area Abbreviation'] = 'MKD'

#### Dropping unecessary columns

In [90]:
clean_fao.drop(columns=['Area Code', 'Item Code', 'Element Code', 'Unit'], inplace=True)

#### Checking latitude and longitude values

Let's check if latitude and longitude contains some non valide values. 

In [91]:
validate_lat_long(clean_fao["latitude"], lat_long=False, lat=True).all()

True

In [92]:
validate_lat_long(clean_fao["longitude"], lat_long=False, lon=True).all()

True

Latitude ranges from -90 to 90,  longitude ranges from -180 to 180

In [93]:
clean_fao[['longitude', 'latitude']].agg([min, max])

,longitude,latitude
min,-172.10,-40.90
max,179.41,64.96


Both max and min of latitude and longitude fall under the accepted range

#### Checking if there are some invalid values in the *Years* columns

Let's check if are there any negative numbers in production

In [94]:
cond = clean_fao.loc[:,'1961':'2013'].agg([min])
cond.T[cond.T['min'] < 0]

,min
2012,-169.00
2013,-246.00


Now let's investigate which country has a negative amount of production for which *Item* and *Element* (food or feed)

In [95]:
clean_fao.loc[clean_fao[['2013', '2012']].idxmin().unique(), ['Area_clean', 'Item', 'Element']]

,Area_clean,Item,Element
10082,Japan,Oats,Food


Negative number in production must be an error of input, just drop the entire row

In [96]:
clean_fao.drop(labels=10082, inplace=True)

#### Checking for missing values

In [97]:
clean_fao.stb.missing(clip_0=True, style=True)

,missing,total,percent
1961,"3,539","21,476",16.48%
1976,"3,539","21,476",16.48%
1989,"3,539","21,476",16.48%
1988,"3,539","21,476",16.48%
1987,"3,539","21,476",16.48%
1962,"3,539","21,476",16.48%
1985,"3,539","21,476",16.48%
1984,"3,539","21,476",16.48%
1983,"3,539","21,476",16.48%
1982,"3,539","21,476",16.48%


Let's create a dataframe containing only the years variables

In [98]:
year_df = clean_fao[years]

First let's drop all the rows that contain all NaN values, if there are any

In [37]:
clean_fao = clean_fao.dropna(how='all')

Since the only NaN values in the dataframe are in the numeric columns, let's fill all the remaining NaN values with 0

In [38]:
clean_fao.fillna(0, inplace=True)

Lastly we  create a list of index of the rows of the years dataframe that has only 0 values

In [39]:
idx_to_drop = year_df.loc[(year_df == 0).all(axis=1)].index

Finally droping those rows

In [40]:
clean_fao = clean_fao.drop(labels=idx_to_drop)

Updating the years dataset with the NaN value dropped

In [41]:
year_df = clean_fao[years]

### Food Production Data

In [42]:
food_pr_df.head()

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),Eutrophying emissions per kilogram (gPO₄eq per kilogram),Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein),Freshwater withdrawals per 1000kcal (liters per 1000kcal),Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),0.10,0.00,0.80,0.20,0.10,0.10,0.10,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),0.30,0.00,0.50,0.10,0.10,0.10,0.00,1.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),0.00,0.00,0.20,0.10,0.00,0.50,0.30,1.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,0.00,0.00,1.40,0.00,0.10,0.10,0.00,1.60,4.28,11.23,8.64,183.91,371.08,482.40,0.95,1.91,2.90,7.60,5.85,"18,786.20","14,450.92","7,162.10"
4,Rice,0.00,0.00,3.60,0.10,0.10,0.10,0.10,4.00,9.51,35.07,49.39,609.98,"3,166.76","2,248.40",1.21,6.27,0.76,2.80,3.94,"49,576.30","69,825.77","13,449.89"


In [43]:
food_pr_df.shape

(43, 23)

In [44]:
food_pr_df.columns

Index(['Food product', 'Land use change', 'Animal Feed', 'Farm', 'Processing',
       'Transport', 'Packging', 'Retail', 'Total_emissions',
       'Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal)',
       'Eutrophying emissions per kilogram (gPO₄eq per kilogram)',
       'Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein)',
       'Freshwater withdrawals per 1000kcal (liters per 1000kcal)',
       'Freshwater withdrawals per 100g protein (liters per 100g protein)',
       'Freshwater withdrawals per kilogram (liters per kilogram)',
       'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)',
       'Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein)',
       'Land use per 1000kcal (m² per 1000kcal)',
       'Land use per kilogram (m² per kilogram)',
       'Land use per 100g protein (m² per 100g protein)',
       'Scarcity-weighted water use per kilogram (liters per kilogram)',
       'Scarcity-weighted water use per 100g protein 

Dropping unecessary columns

In [45]:
to_drop = [
       'Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal)',
       'Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein)',
       'Freshwater withdrawals per 1000kcal (liters per 1000kcal)',
       'Freshwater withdrawals per 100g protein (liters per 100g protein)',
       'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)',
       'Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein)',
       'Land use per 1000kcal (m² per 1000kcal)',
       'Land use per 100g protein (m² per 100g protein)',
       'Scarcity-weighted water use per 100g protein (liters per 100g protein)',
       'Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)'
       ]
food_pr_df.drop(to_drop, axis=1, inplace=True)

Removing missing values

In [46]:
food_pr_df.dropna(
    how='all', 
    subset=['Eutrophying emissions per kilogram (gPO₄eq per kilogram)', 
            'Freshwater withdrawals per kilogram (liters per kilogram)', 
            'Land use per kilogram (m² per kilogram)',
            'Scarcity-weighted water use per kilogram (liters per kilogram)'
            ],
    inplace=True)

# Data Exploration/Data Transformation

Calculating the total production for item from 1961 to 2012

In [47]:
clean_fao['Total Production'] = clean_fao[years].sum(axis=1)

In [48]:
# filtering datasets for 'Food' and 'Feed' 
cond_1 = clean_fao['Element'] == 'Food'
cond_2 = clean_fao['Element'] == 'Feed'

#creating those datasets
Food = clean_fao.loc[cond_1, :]
Feed = clean_fao.loc[cond_2, :]

Calculating the mean of production from the first year of production of the item

In [ ]:
#tentativo fallimentare di trovare una media a partire dal primo valore non nullo per riga della produzione
# def func(row):
#     a = np.array(year_df.loc[row, :])
#     ind = np.nonzero(a > 0)[0][0]
#     if ind == 0:
#         mean = clean_fao['Total Production'] / len(a)
#         return mean
#     elif ind > 0 and ind < len(a):
#         mean = clean_fao['Total Production'] / (len(a) - (ind-1))
#         return mean
#     elif ind == len(a):
#         mean = 0
#         return mean        
             
# clean_fao['mean prod'] = year_df.apply(func, raw=True, axis=1)
        

In [49]:
food_pr_df.describe()

,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per kilogram (gPO₄eq per kilogram),Freshwater withdrawals per kilogram (liters per kilogram),Land use per kilogram (m² per kilogram),Scarcity-weighted water use per kilogram (liters per kilogram)
count,38.00,38.00,38.00,38.00,38.00,38.00,38.00,38.00,38.00,38.00,38.00,38.00
mean,1.38,0.45,3.65,0.26,0.21,0.27,0.06,6.27,46.14,932.61,29.26,"36,607.43"
std,3.56,0.91,7.44,0.38,0.16,0.36,0.10,11.05,82.81,"1,297.00",78.49,"56,891.28"
min,-2.10,0.00,0.10,0.00,0.10,0.00,0.00,0.20,0.69,0.00,0.33,0.00
25%,0.00,0.00,0.33,0.00,0.10,0.03,0.00,0.80,3.75,105.50,1.11,"3,325.07"
50%,0.15,0.00,1.20,0.10,0.10,0.10,0.00,2.00,11.46,417.10,6.87,"14,533.05"
75%,0.85,0.00,2.25,0.30,0.28,0.30,0.08,6.00,45.84,"1,340.38",14.92,"35,960.18"
max,16.30,2.90,39.40,1.30,0.80,1.60,0.30,59.60,365.29,"5,605.20",369.81,"229,889.80"


##### Top 10 product for emissions 

In [50]:
fig = px.histogram(food_pr_df, x=food_pr_df['Eutrophying emissions per kilogram (gPO₄eq per kilogram)'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()


75% of the emissions are under 50 gPO₄eq per kilogram

Since we want to know the top product that contributes to the Eutrophyng emissions per kilogram, we first find a threshold above which set the analysys.
The threshold is set above the 3rd percentile: 

In [51]:
cond = food_pr_df['Eutrophying emissions per kilogram (gPO₄eq per kilogram)'] > np.percentile(food_pr_df['Eutrophying emissions per kilogram (gPO₄eq per kilogram)'], 75)
food_pr_df.loc[cond, ['Food product', 'Eutrophying emissions per kilogram (gPO₄eq per kilogram)']].sort_values(by='Eutrophying emissions per kilogram (gPO₄eq per kilogram)', ascending=False)

,Food product,Eutrophying emissions per kilogram (gPO₄eq per kilogram)
34,Beef (dairy herd),365.29
33,Beef (beef herd),301.41
41,Fish (farmed),235.12
31,Coffee,110.52
39,Cheese,98.37
35,Lamb & Mutton,97.13
32,Dark Chocolate,87.08
36,Pig Meat,76.38
17,Sunflower Oil,50.66
37,Poultry Meat,48.70


In [52]:
top_emissions = food_pr_df.sort_values(by='Eutrophying emissions per kilogram (gPO₄eq per kilogram)', ascending=False)[:10]['Food product'].to_list()

Top 10 product for freshwater withdrawals

In [53]:
fig = px.histogram(food_pr_df, x=food_pr_df['Freshwater withdrawals per kilogram (liters per kilogram)'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()

In [54]:
cond = food_pr_df['Freshwater withdrawals per kilogram (liters per kilogram)'] > np.percentile(food_pr_df['Freshwater withdrawals per kilogram (liters per kilogram)'], 75)
food_pr_df.loc[cond, ['Food product', 'Freshwater withdrawals per kilogram (liters per kilogram)']].sort_values(by='Freshwater withdrawals per kilogram (liters per kilogram)', ascending=False)

,Food product,Freshwater withdrawals per kilogram (liters per kilogram)
39,Cheese,"5,605.20"
11,Nuts,"4,133.80"
41,Fish (farmed),"3,691.30"
34,Beef (dairy herd),"2,714.30"
4,Rice,"2,248.40"
19,Olive Oil,"2,141.80"
12,Groundnuts,"1,852.30"
35,Lamb & Mutton,"1,802.80"
36,Pig Meat,"1,795.80"
33,Beef (beef herd),"1,451.20"


In [55]:
top_water = food_pr_df.sort_values(by='Freshwater withdrawals per kilogram (liters per kilogram)', ascending=False)[:10]['Food product'].to_list()


Top 10 prodcuts for land use

In [56]:
fig = px.histogram(food_pr_df, x=food_pr_df['Land use per kilogram (m² per kilogram)'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()

In [57]:
cond = food_pr_df['Land use per kilogram (m² per kilogram)'] > np.percentile(food_pr_df['Land use per kilogram (m² per kilogram)'], 75)
food_pr_df.loc[cond, ['Food product', 'Land use per kilogram (m² per kilogram)']].sort_values(by='Land use per kilogram (m² per kilogram)', ascending=False)

,Food product,Land use per kilogram (m² per kilogram)
35,Lamb & Mutton,369.81
33,Beef (beef herd),326.21
39,Cheese,87.79
32,Dark Chocolate,68.96
34,Beef (dairy herd),43.24
19,Olive Oil,26.31
31,Coffee,21.62
17,Sunflower Oil,17.66
36,Pig Meat,17.36
9,Other Pulses,15.57


In [58]:
top_land = food_pr_df.sort_values(by='Land use per kilogram (m² per kilogram)', ascending=False)[:10]['Food product'].to_list()

Top 10 products for scarcity-weighted water use

In [59]:
fig = px.histogram(food_pr_df, x=food_pr_df['Scarcity-weighted water use per kilogram (liters per kilogram)'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()

In [60]:
cond = food_pr_df['Scarcity-weighted water use per kilogram (liters per kilogram)'] > np.percentile(food_pr_df['Scarcity-weighted water use per kilogram (liters per kilogram)'], 75)
food_pr_df.loc[cond, ['Food product', 'Scarcity-weighted water use per kilogram (liters per kilogram)']].sort_values(by='Scarcity-weighted water use per kilogram (liters per kilogram)', ascending=False)

,Food product,Scarcity-weighted water use per kilogram (liters per kilogram)
11,Nuts,"229,889.80"
39,Cheese,"180,850.60"
19,Olive Oil,"177,480.20"
35,Lamb & Mutton,"141,925.00"
34,Beef (dairy herd),"119,805.20"
36,Pig Meat,"66,867.40"
12,Groundnuts,"61,797.90"
4,Rice,"49,576.30"
41,Fish (farmed),"41,572.20"
17,Sunflower Oil,"36,369.40"


In [61]:
top_scarc = food_pr_df.sort_values(by='Scarcity-weighted water use per kilogram (liters per kilogram)', ascending=False)[:10]['Food product'].to_list()

Top 10 products for Green House emissions

In [62]:
fig = px.histogram(food_pr_df, x=food_pr_df['Total_emissions'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()

In [63]:
cond = food_pr_df['Total_emissions'] > np.percentile(food_pr_df['Total_emissions'], 75)
food_pr_df.loc[cond, ['Food product', 'Total_emissions']].sort_values(by='Total_emissions', ascending=False)

,Food product,Total_emissions
33,Beef (beef herd),59.60
35,Lamb & Mutton,24.50
39,Cheese,21.20
34,Beef (dairy herd),21.10
32,Dark Chocolate,18.70
31,Coffee,16.50
16,Palm Oil,7.60
36,Pig Meat,7.20
37,Poultry Meat,6.10


In [64]:
top_emiss = food_pr_df.sort_values(by='Total_emissions', ascending=False)[:10]['Food product'].to_list()

In [66]:
clean_fao.sort_values(by='Total Production', ascending=False).loc[:, ['Area_clean', 'Item', 'Total Production']][:20]

,Area_clean,Item,Total Production
4249,People's Republic of China,Vegetables,"9,046,594.00"
4237,People's Republic of China,Cereals - Excluding Beer,"8,678,866.00"
4186,People's Republic of China,"Vegetables, Other","7,962,498.00"
20506,United States of America,Cereals - Excluding Beer,"7,408,504.00"
9139,Republic of India,Cereals - Excluding Beer,"6,675,867.00"
20399,United States of America,Maize and products,"6,068,125.00"
4239,People's Republic of China,Starchy Roots,"4,623,978.00"
4123,People's Republic of China,Rice (Milled Equivalent),"4,271,067.00"
4236,People's Republic of China,Cereals - Excluding Beer,"4,084,713.00"
4121,People's Republic of China,Wheat and products,"3,521,951.00"


In [ ]:
# def convert(row):
#     cn_code = pc.country_alpha3_to_country_alpha2(row['Area Abbreviation'])
#     conti_code = pc.country_alpha2_to_continent_code(cn_code)
#     return conti_code

# clean_fao['continent'] = clean_fao.apply(convert, axis=1)
# clean_fao

# country_code = pc.country_name_to_country_alpha2(["China", "Italy"], cn_name_format="default")
# print(country_code)
# continent_name = pc.country_alpha2_to_continent_code(country_code)
# print(continent_name)

Creating a column with the relative continent for each state

In [69]:
converter = coco.CountryConverter()
def func(row):
    continent = converter.convert(names = row['Area Abbreviation'], src = 'ISO3', to = 'continent')
    return continent

clean_fao['continent'] = clean_fao.apply(func, axis=1)

,Area Abbreviation,Item,Element,latitude,longitude,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,Area_clean,Total Production,continent
0,AFG,Wheat and products,Food,33.94,67.71,"1,928.00","1,904.00","1,666.00","1,950.00","2,001.00","1,808.00","2,053.00","2,045.00","2,154.00","1,819.00","1,963.00","2,215.00","2,310.00","2,335.00","2,434.00","2,512.00","2,282.00","2,454.00","2,443.00","2,129.00","2,133.00","2,068.00","1,994.00","1,851.00","1,791.00","1,683.00","2,194.00","1,801.00","1,754.00","1,640.00","1,539.00","1,582.00","1,840.00","1,855.00","1,853.00","2,177.00","2,343.00","2,407.00","2,463.00","2,600.00","2,668.00","2,776.00","3,095.00","3,249.00","3,486.00","3,704.00","4,164.00","4,252.00","4,538.00","4,605.00","4,711.00","4,810.00","4,895.00",Islamic Republic of Afghanistan,"132,926.00",Asia
1,AFG,Rice (Milled Equivalent),Food,33.94,67.71,183.00,183.00,182.00,220.00,220.00,195.00,231.00,235.00,238.00,213.00,205.00,233.00,246.00,246.00,255.00,263.00,235.00,254.00,270.00,259.00,248.00,217.00,217.00,197.00,186.00,200.00,193.00,202.00,191.00,199.00,197.00,249.00,218.00,260.00,319.00,254.00,326.00,347.00,270.00,372.00,411.00,448.00,460.00,419.00,445.00,546.00,455.00,490.00,415.00,442.00,476.00,425.00,422.00,Islamic Republic of Afghanistan,"15,282.00",Asia
2,AFG,Barley and products,Feed,33.94,67.71,76.00,76.00,76.00,76.00,76.00,75.00,71.00,72.00,73.00,74.00,71.00,70.00,72.00,76.00,77.00,80.00,60.00,65.00,64.00,64.00,60.00,55.00,53.00,51.00,48.00,46.00,46.00,47.00,46.00,43.00,43.00,40.00,50.00,46.00,41.00,44.00,50.00,48.00,43.00,26.00,29.00,70.00,48.00,58.00,236.00,262.00,263.00,230.00,379.00,315.00,203.00,367.00,360.00,Islamic Republic of Afghanistan,"5,190.00",Asia
3,AFG,Barley and products,Food,33.94,67.71,237.00,237.00,237.00,238.00,238.00,237.00,225.00,227.00,230.00,234.00,223.00,219.00,225.00,240.00,244.00,255.00,185.00,203.00,198.00,202.00,189.00,174.00,167.00,160.00,151.00,145.00,145.00,148.00,145.00,135.00,132.00,120.00,155.00,143.00,125.00,138.00,159.00,154.00,141.00,84.00,83.00,122.00,144.00,185.00,43.00,44.00,48.00,62.00,55.00,60.00,72.00,78.00,89.00,Islamic Republic of Afghanistan,"8,529.00",Asia
4,AFG,Maize and products,Feed,33.94,67.71,210.00,210.00,214.00,216.00,216.00,216.00,235.00,232.00,236.00,200.00,201.00,216.00,228.00,231.00,234.00,240.00,228.00,234.00,228.00,226.00,210.00,199.00,192.00,182.00,173.00,170.00,154.00,148.00,137.00,144.00,126.00,90.00,141.00,150.00,159.00,108.00,90.00,99.00,72.00,35.00,48.00,89.00,63.00,120.00,208.00,233.00,249.00,247.00,195.00,178.00,191.00,200.00,200.00,Islamic Republic of Afghanistan,"9,451.00",Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21470,ZWE,Eggs,Food,-19.02,29.15,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,7.00,7.00,7.00,7.00,8.00,8.00,8.00,9.00,9.00,9.00,9.00,9.00,9.00,10.00,9.00,8.00,8.00,9.00,11.00,12.00,14.00,14.00,14.00,14.00,13.00,15.00,15.00,13.00,14.00,14.00,16.00,18.00,11.00,19.00,15.00,18.00,18.00,21.00,22.00,27.00,27.00,24.00,24.00,25.00,Republic of Zimbabwe,652.00,Africa
21471,ZWE,Milk - Excluding Butter,Feed,-19.02,29.15,12.00,12.00,12.00,11.00,11.00,11.00,11.00,12.00,14.00,13.00,17.00,19.00,21.00,21.00,20.00,23.00,22.00,17.00,20.00,18.00,21.00,17.00,14.00,14.00,13.00,12.00,10.00,9.00,6.00,6.00,4.00,10.00,20.00,13.00,38.00,40.00,28.00,35.00,35.00,36.00,36.00,28.00,21.00,21.00,21.00,21.00,21.00,21.00,23.00,25.00,25.00,30.00,31.00,Republic of Zimbabwe,"1,022.00",Africa
21472,ZWE,Milk - Excluding Butter,Food,-19.02,29.15,230.00,232.00,234.00,238.00,234.00,231.00,231.00,248.00,264.00,255.00,306.00,322.00,343.00,337.00,358.00,379.00,397.

# Data Visualization

In [ ]:
# dataset popolazione, dataset km2 di terra
# normalizzare produzione con popolazione
# normalizzare uso terra con km2
# trovare i top 10 prodotti che consumano piu risorse
# trovare i top 10 stati che producono quei prodotti